# Notebook for ingesting JSON from BPA REST API

BPA API is built on CKAN - docs: https://docs.ckan.org/en/latest/api/

Base URL for BPA is https://data.bioplatforms.com/api/3

Ingest using wildcard search: https://data.bioplatforms.com/api/3/action/package_search?q=*:*&rows=1000 

Note script will need to paginate 1000 records at a time (48,438 results)

In [17]:
import requests
import pandas as pd
from pandas.io.json import json_normalize 
import numpy as nmp

url = "https://data.bioplatforms.com/api/3/action/package_search?q=*:*&rows="
pagesize = 1000
total = requests.get(f"{url}0").json()['result']['count']
totalPages = int(nmp.ceil(total / pagesize)) + 1 # range needs an extra 1
print("Total:", total, totalPages)
response = requests.get(f"{url}{pagesize}").json()['result']['results']
# df = pd.DataFrame.from_dict(data)
api_dataset = response

for page in range (1, totalPages):
    # print("api_dataset size", len(api_dataset))
    print("Paginating:", page, "start=", (page * pagesize))
    # print("URL", f"{url}{pagesize}&start={(page * pagesize)}")
    response = requests.get(f"{url}{pagesize}&start={(page * pagesize)}").json()['result']['results']
    api_dataset.extend(response)

print("api_dataset size", len(api_dataset))
df = json_normalize(api_dataset) # transformation to dataframe via normalize function

df.info()

Total: 48438 50
Paginating: 1 start= 1000
Paginating: 2 start= 2000
Paginating: 3 start= 3000
Paginating: 4 start= 4000
Paginating: 5 start= 5000
Paginating: 6 start= 6000
Paginating: 7 start= 7000
Paginating: 8 start= 8000
Paginating: 9 start= 9000
Paginating: 10 start= 10000
Paginating: 11 start= 11000
Paginating: 12 start= 12000
Paginating: 13 start= 13000
Paginating: 14 start= 14000
Paginating: 15 start= 15000
Paginating: 16 start= 16000
Paginating: 17 start= 17000
Paginating: 18 start= 18000
Paginating: 19 start= 19000
Paginating: 20 start= 20000
Paginating: 21 start= 21000
Paginating: 22 start= 22000
Paginating: 23 start= 23000
Paginating: 24 start= 24000
Paginating: 25 start= 25000
Paginating: 26 start= 26000
Paginating: 27 start= 27000
Paginating: 28 start= 28000
Paginating: 29 start= 29000
Paginating: 30 start= 30000
Paginating: 31 start= 31000
Paginating: 32 start= 32000
Paginating: 33 start= 33000
Paginating: 34 start= 34000
Paginating: 35 start= 35000
Paginating: 36 start= 

/var/folders/sw/fck698gd1wv_56c640ksf4r0009m0h/T/ipykernel_92219/90665411.py:23: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  df = json_normalize(api_dataset) # transformation to dataframe via normalize function


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48438 entries, 0 to 48437
Columns: 976 entries, library_ng_ul to license_url
dtypes: bool(3), int64(2), object(971)
memory usage: 359.7+ MB


In [18]:
df.head()

,library_ng_ul,data_context,method_of_determination,ancillary_notes,dna_treatment,sequencing_facility,sequence_data_type,library_type,barcode_id,title,...,library_id_description,dataset_context,reference_genome_link,update_frequency,theme,data_homepage,language,data_portal,doi,license_url
0,2.21,Population genetics,,,,AGRF Melbourne,illumina-ddrad,ddRAD,,TSI Genomics ddRAD 102.100.100/358765 library,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Population genetics,,,,AGRF Melbourne,illumina-ddrad,ddRAD,,TSI Genomics ddRAD 102.100.100/358765 library,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Population genetics,,NaN,,AGRF Melbourne,illumina-ddrad,ddRAD,,TSI Genomics ddRAD 102.100.100/358764 library,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.38,Transcriptome,NaN,NaN,N/A,UNSW,illumina-shortread,Illumina-transcriptomics,NaN,TSI Illumina FastQ 102.100.100/408040 HHV7HDRX2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7.44,Transcriptome,Dissection,,N/A,UNSW,illumina-shortread,Illumina-transcriptomics,,TSI Illumina FastQ 102.100.100/408039 HHV7HDRX2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.to_csv("/data/arga-data/bpa_export.csv", index=False)